In [20]:
import os
import pandas as pd
from nfstream import NFStreamer
from glob import glob
from tqdm.notebook import tqdm
from PIL import Image


In [2]:
PCAP_PATHS = glob(
    '/media/research/TrungHieuOnly/Hieu/Datasets/ISCX-VPN-NonVPN-2016/NonVPN-PCAPs/*')
LABELS = ['aim', 'email', 'facebook', 'ftps', 'gmail',
          'hangout', 'icq', 'netflix', 'scp', 'sftp',
          'skype', 'spotify', 'vimeo', 'voipbuster', 'youtube']
UNUSED_COL = ['id', 'src_ip', 'src_mac', 'src_oui', 'dst_ip', 'dst_mac', 'dst_oui', 'splt_direction', 'splt_ps', 'splt_piat_ms', 'application_name', 'application_category_name',
              'application_is_guessed', 'application_confidence', 'requested_server_name', 'client_fingerprint', 'server_fingerprint', 'user_agent', 'content_type']


In [3]:
for path in tqdm(PCAP_PATHS):
    file_name = os.path.basename(path)
    dst_path = 'data/flow/' + os.path.splitext(file_name)[0] + '.csv'
    df = NFStreamer(source=path, statistical_analysis=True,
                    splt_analysis=10).to_pandas()
    df = df.drop(columns=UNUSED_COL)
    for label in LABELS:
        if file_name.lower().startswith(label):
            df['label'] = label
    df.to_csv(dst_path, index=False)


  0%|          | 0/109 [00:00<?, ?it/s]

/home/research/.pyenv/versions/3.9.0/lib/python3.9/site-packages/nfstream/streamer.py:513: DtypeWarning: Columns (86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(temp_file_path)
/home/research/.pyenv/versions/3.9.0/lib/python3.9/site-packages/nfstream/streamer.py:513: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(temp_file_path)


In [7]:
src_paths = glob('data/flow/*')
for label in LABELS:
    file_list = []
    for path in src_paths:
        file_name = os.path.basename(path)
        if file_name.lower().startswith(label):
            file_list.append(pd.read_csv(path))
    merged_df = pd.concat(file_list, ignore_index=True)
    print(label, len(merged_df))


aim 431
email 8362
facebook 106923
ftps 923
gmail 463
hangout 101979
icq 447
netflix 391
scp 11944
sftp 225
skype 127067
spotify 365
vimeo 467
voipbuster 3120
youtube 947


In [17]:
labels = ['facebook', 'hangout', 'skype']
df = pd.DataFrame()
for path in src_paths:
    file_name = os.path.basename(path)
    other = True
    for label in labels:
        if file_name.lower().startswith(label):
            df = pd.concat([df, pd.read_csv(path)], ignore_index=True)
            other = False
            break
    if other:
        df_other = pd.read_csv(path)
        df_other['label'] = 'other'
        df = pd.concat([df, df_other], ignore_index=True)
df.to_csv('nfstream_data.csv', index=False)


In [18]:
data = pd.read_csv('nfstream_data.csv')


In [19]:
data['label'].value_counts()


skype       127067
facebook    106923
hangout     101979
other        28085
Name: label, dtype: int64

In [24]:
num = {
    'facebook': 0,
    'hangout': 0,
    'skype': 0,
    'other': 0,
}
for index, row in tqdm(data.iterrows()):
    x = row.values[:-1].astype('float')
    label = row.values[-1]
    num[label] += 1
    if (n := num[label]) <= 10000:
        image = Image.fromarray(x, mode='RGBA')
        image.save(f'data/rgba/{label}/{n}.png')


0it [00:00, ?it/s]